In [123]:
import math
import random
import time

import pandas as pd
from numpy import mean


root = '/home/jimge/public/CAP5610/hw1/'
iris = pd.read_csv(root + 'iris.data', names=['seplen', 'sepwid', 'petlen', 'petwid', 'class'])
iris = iris.values.tolist()
iris = [{'pt': tuple(x[:-1]), 'label': x[-1]} for x in iris]
iris_class = set([x['label'] for x in iris])
iris_class

{'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'}

In [79]:
'''
Data for task 1
'''
data = [
    { 'pt': (3, 5), 'label': 'X1'},
    { 'pt': (3, 4), 'label': 'X2'},
    { 'pt': (2, 8), 'label': 'X3'},
    { 'pt': (2, 3), 'label': 'X4'},
    { 'pt': (6, 2), 'label': 'X5'},
    { 'pt': (6, 4), 'label': 'X6'},
    { 'pt': (7, 3), 'label': 'X7'},
    { 'pt': (7, 4), 'label': 'X8'},
    { 'pt': (8, 5), 'label': 'X9'},
    { 'pt': (7, 6), 'label': 'X10'}
]

In [344]:
'''
Implementations for tasks 1 and 2
'''
def sumsq(pt0, pt1):
    '''
    Computes the sum of squares of two points
    '''
    return sum([(x-y)**2 for (x,y) in zip(pt0, pt1)])
    
def mandist(pt0, pt1):
    '''
    Compute the Manhattan distance between two points
    '''
    return sum([abs(x-y) for (x,y) in zip(pt0, pt1)])

def eucdist(pt0, pt1):
    '''
    Compute the Euclidean distance between two points
    '''
    return math.sqrt(sumsq(pt0, pt1))

def dot(pt0, pt1):
    '''
    Compute the dot product of two vectors
    '''
    return sum([x*y for (x,y) in zip(pt0, pt1)])

def mag(pt0):
    '''
    Compute the magnitude of a vector
    '''
    return math.sqrt(sum([x*x for x in pt0]))

def cosdist(pt0, pt1):
    '''
    Compute the cosine distance between two points
    '''
    return 1 - dot(pt0, pt1) / (mag(pt0) * mag(pt1))

def jacdist(pt0, pt1):
    '''
    Compute the Jaccard distance between two points
    '''
    mins = sum([min(x,y) for (x,y) in zip(pt0, pt1)])
    maxes = sum([max(x,y) for (x,y) in zip(pt0, pt1)])
    return 1 - mins / maxes

def calc_centroid(vecs, dim):
    '''
    Given a list of equal size vectors, find the centroid
    '''
    if len(vecs) == 0:
        return [float('inf') for i in range(0, dim)]
    
    s = vecs[0]
    assert len(s) == dim
    for v in vecs[1:]:
        assert len(v) == len(s)
        s = [x+y for (x, y) in zip(s, v)]
    invl = 1 / len(vecs)
    m = [x*invl for x in s]
    return m

def kpass(data, metric, K, centroids = None, maxiter = None):
    '''
    Perform one k-means pass
    '''
    if centroids == None or len(centroids) < K:
        # If there aren't centroids, pick some from the data
        random.seed(time.time())
        centroids = random.sample(data, K)
        centroids = [sample['pt'] for sample in centroids]
        
    # dimension of the samples
    dim = len(data[0]['pt'])
    
    # compute the clusters
    clusters = [[] for c in centroids]
    
    for sample in data:
        dists = [(i, metric(c, sample['pt'])) for (i, c) in enumerate(centroids)]
        best = min(dists, key = lambda x: x[1])
        clusters[best[0]].append(sample)
        
    # compute the new centroids
    clus_points = [[s['pt'] for s in clus] for clus in clusters]
    centroids = [calc_centroid(clus, dim) for clus in clus_points]
    ss = sum([sum([sumsq(pt, ce) for pt in cl]) for (cl, ce) in zip(clus_points, centroids)])
    
    return {'clusters': clusters, 'centroids': centroids, 'ss': ss}

TERM_CENTROIDS = 1
TERM_ITERS = 2
TERM_SSE = 4

def kmeans(data, metric, K, centroids = None, term = TERM_CENTROIDS, maxiter = 100):   
    '''
    Run k-means until the given termination condition(s)
    '''
    iters = 0
    prevk = None
    while True:
        iters += 1
        k = kpass(data, metric, K, centroids)
        
        # terminate if the centroids didn't change
        if (term & TERM_CENTROIDS) != 0 and centroids == k['centroids']:
            break
            
        # terminate if we hit the iteration cap
        if (term & TERM_ITERS) != 0 and iters == maxiter:
            break
            
        # term if SSE rises or doesn't change (per class discussion)
        if (term & TERM_SSE) != 0 and prevk != None and k['ss'] >= prevk['ss']:
            k = prevk
            break

        prevk = k
        centroids = k['centroids']
        
    k['iterations'] = iters
    return k

def cluslabels(clusters):
    '''
    Return lists of labels for all clusters
    '''
    return [[x['label'] for x in c] for c in clusters]

methods = {
    'euclidean': eucdist,
    'cosine': cosdist,
    'jacard': jacdist
}

In [345]:
'''
Task 1 - k-means on simple test data
'''

K = 2

# Q1 
centroids = [(4, 6), (5, 4)]
print('Q1 Manhattan distance')
print(kpass(data, mandist, K, centroids=centroids)['centroids'])
print(cluslabels(kmeans(data, mandist, K, centroids=centroids)['clusters']))

# Q2 
centroids = [(4, 6), (5, 4)]
print('\nQ2 Euclidean distance')
print(kpass(data, eucdist, K, centroids=centroids)['centroids'])
print(cluslabels(kmeans(data, eucdist, K, centroids=centroids)['clusters']))

#Q3
centroids = [(3, 3), (8, 3)]
print('\nQ3 Manhattan distance')
print(kpass(data, mandist, K, centroids=centroids)['centroids'])
print(cluslabels(kmeans(data, mandist, K, centroids=centroids)['clusters']))

# Q4
centroids = [(3, 2), (4, 8)]
print('\nQ4 Manhattan distance')
print(kpass(data, mandist, K, centroids=centroids)['centroids'])
print(cluslabels(kmeans(data, mandist, K, centroids=centroids)['clusters']))


Q1 Manhattan distance
[[4.0, 6.333333333333333], [5.571428571428571, 3.571428571428571]]
[['X1', 'X3', 'X10'], ['X2', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']]

Q2 Euclidean distance
[[2.5, 6.5], [5.75, 3.875]]
[['X1', 'X2', 'X3', 'X4'], ['X5', 'X6', 'X7', 'X8', 'X9', 'X10']]

Q3 Manhattan distance
[[2.5, 5.0], [6.833333333333333, 4.0]]
[['X1', 'X2', 'X3', 'X4'], ['X5', 'X6', 'X7', 'X8', 'X9', 'X10']]

Q4 Manhattan distance
[[4.857142857142857, 3.571428571428571], [5.666666666666666, 6.333333333333333]]
[['X1', 'X2', 'X4', 'X5', 'X6', 'X7', 'X8'], ['X3', 'X9', 'X10']]


In [329]:
'''
Task 2 Q1
'''
K = len(iris_class)
trials = 100
totals = dict(zip(methods.keys(), len(methods.keys()) * [0]))

for t in range(0, trials):
    for methname, methfunc in methods.items():
        totals[methname] += kmeans(iris, methfunc, K)['ss']
    
print('\n'.join(['%-12s %8.4f' % (x[0], x[1] / trials) for x in totals.items()]))

euclidean     95.6821
cosine       107.4625
jacard        97.5731


In [330]:
'''
Task 2 Q2
'''
def majcount(clus):
    '''
    Return the label with the highest occurance in this cluster
    '''
    if len(clus) == 0:
        return ('', 0)
    counts = dict(zip(iris_class, len(iris_class) * [0]))
    for l in [x['label'] for x in clus]:
        counts[l] += 1
    counts = list(counts.items())
    return max(counts, key = lambda x: x[1])

def accuracy(results):
    '''
    Return the overall accuracy
    '''
    # if we define the cluster label as the majority count, then by definition the
    # number correct in that cluster IS the majority count
    correct = sum([majcount(cl)[1] for cl in results['clusters']])
    return correct / len(iris)

K = len(iris_class)
trials = 100
totals = dict(zip(methods.keys(), len(methods.keys()) * [0]))

for t in range(0, trials):
    for methname, methfunc in methods.items():
        totals[methname] += accuracy(kmeans(iris, methfunc, K))
    
print('\n'.join(['%-12s %8.6f' % (x[0], x[1] / trials) for x in totals.items()]))

euclidean    0.860933
cosine       0.891600
jacard       0.860133


In [316]:
'''
Task 2 Q3
'''
K = len(iris_class)
trials = 100
totals = dict(zip(methods.keys(), [[0,0] for x in methods.keys()]))

for t in range(0, trials):
    for methname, methfunc in methods.items():
        start = time.time()
        totals[methname][0] += kmeans(iris, methfunc, K)['iterations']
        totals[methname][1] += time.time() - start
 
print('\n'.join(['%-12s %8.4f %8.4fms' % (x[0], x[1][0] / trials, 1000 * x[1][1] / trials) for x in totals.items()]))

euclidean      7.1200   5.9380ms
cosine         5.6600   6.8083ms
jacard         5.5500   7.4648ms


In [338]:
'''
Task 2 Q4
'''
tests = {
    'centroids': TERM_CENTROIDS,
    'iterations': TERM_ITERS,
    'sse': TERM_SSE
}

K = len(iris_class)
trials = 100

for (testname, test) in tests.items():
    totals = dict(zip(methods.keys(), [[0,0,0] for x in methods.keys()]))
    for t in range(0, trials):
        for methname, methfunc in methods.items():
            start = time.time()
            k = kmeans(iris, methfunc, K, term=test, maxiter=100)
            totals[methname][0] += k['ss']
            totals[methname][1] += k['iterations']
            totals[methname][2] += time.time() - start
 
    print(testname)
    print('  method       ---SS--- --ITER-- ---TIME---')
    print('  ' + '\n  '.join(['%-12s %8.4f %8.4f %8.4fms' % (x[0], x[1][0] / trials, x[1][1] / trials, 1000 * x[1][2] / trials) for x in totals.items()]))
    

centroids
  method       ---SS--- --ITER-- ---TIME---
  euclidean     93.1280   7.2100   5.7210ms
  cosine       107.7838   6.0100   7.1862ms
  jacard        97.1745   5.8900   7.7068ms
iterations
  method       ---SS--- --ITER-- ---TIME---
  euclidean     95.0155 100.0000  76.7925ms
  cosine       106.9400 100.0000 116.2863ms
  jacard        96.2830 100.0000 130.1931ms
sse
  method       ---SS--- --ITER-- ---TIME---
  euclidean     88.0359   7.1600   5.5638ms
  cosine       109.3572   4.8900   5.7016ms
  jacard        95.4317   4.5000   5.8427ms


In [346]:
'''
Task 3
'''
red = [(4.7, 3.2), (4.9, 3.1), (5.0, 3.0), (4.6, 2.9)]
blue = [(5.9, 3.2), (6.7, 3.1), (6.0, 3.0), (6.2, 2.8)]

dists = []
for r in red:
    for b in blue:
        dists.append(eucdist(r, b))

print("min  %.4f" % min(dists))
print("max  %.4f" % max(dists))
print("mean %.4f" % mean(dists))

min  0.9220
max  2.1095
mean 1.4129
